In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import requests
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import os
import csv

In [9]:
# Enable verbose logging
options = webdriver.ChromeOptions()
options.add_argument("--verbose")

driver = webdriver.Chrome(options=options)
driver.get('https://collegedunia.com/usa/business-analytics/master-universities')

# Pause to allow page elements to load
time.sleep(5)

def close_popup(driver):
    try:
        # Set a timeout duration for the wait
        timeout = 10  # seconds
        
        # Wait until the pop-up is present
        time.sleep(10)
        close_button = driver.find_element(By.CLASS_NAME, "lead-close-icon")
        
        # Once clickable, click the close button to close the pop-up
        close_button.click()
        print("Pop-up closed.")
    except TimeoutException:
        print("No pop-up appeared within the timeout period.")
    except NoSuchElementException:
        print("No pop-up found.")
    except ElementClickInterceptedException:
        print("Pop-up close button not clickable.")

close_popup(driver)

# Initialize the page number
page_number = 1
save_directory = os.path.expanduser('~/Desktop/Homework/DDR/Final Project')  # Corrected path

while True:
    # Define the full path to the file
    file_path = os.path.join(save_directory, f"page_{page_number}.html")
    
    # Ensure the directory exists
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    
    # Pause before saving the page
    time.sleep(20)
    
    # Save the current page's HTML to the specified file path
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(driver.page_source)
        print(f"Saved: {file_path}")
    
    # Pause before finding the 'next' button
    time.sleep(5)

    try:
        # Find the 'next page' button by its 'rel' attribute
        next_page_button = driver.find_element(By.CSS_SELECTOR, "a[rel='next']")

        # Scroll the button into view and try clicking
        driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
        # Pause before clicking the 'next' button
        time.sleep(5)
        
        next_page_button.click()
        
        # Pause to wait for the next page to load
        time.sleep(5)
        
        # Increment the page number for the next iteration of the loop
        page_number += 1

    except NoSuchElementException:
        print("Next page button does not exist. Exiting the loop.")
        break
    except ElementClickInterceptedException:
        print("Next page button was not clickable. There might be an overlay or the button is obscured.")
        break

# Close the driver
driver.quit()

Pop-up closed.
Saved: /Users/xinyutu/Desktop/Homework/DDR/Final Project/page_1.html
Saved: /Users/xinyutu/Desktop/Homework/DDR/Final Project/page_2.html
Saved: /Users/xinyutu/Desktop/Homework/DDR/Final Project/page_3.html
Saved: /Users/xinyutu/Desktop/Homework/DDR/Final Project/page_4.html
Next page button does not exist. Exiting the loop.


In [10]:
for filename in sorted(os.listdir(save_directory)):  # Sort to ensure processing order
    if filename.endswith(".html"):
        filepath = os.path.join(save_directory, filename)
        with open(filepath, 'r', encoding='utf-8') as file:
            html = file.read()

        print(f"Read {filename}.")

        # Process the HTML content
        soup = BeautifulSoup(html, 'html.parser')

Read page_1.html.
Read page_2.html.
Read page_3.html.
Read page_4.html.


In [69]:

all_school_info = []

for filename in sorted(os.listdir(save_directory)):
    if filename.endswith(".html"):
        filepath = os.path.join(save_directory, filename)
        with open(filepath, 'r', encoding='utf-8') as file:
            html = file.read()

        print(f"Read {filename}.")

        # Process the HTML content
        soup = BeautifulSoup(html, 'html.parser')
        # Find all 'a' tags with the 'data-ga-event-category' attribute set to 'info'
        a_tags = soup.find_all('a', {'data-ga-event-category': 'info'})

        for a_tag in a_tags:
            school_name = a_tag.get_text(strip=True)
            if school_name:  # Only proceed if the school name is not empty
                school_info = {'name': school_name}

                # Find the parent 'tr' of the 'a' tag, as 'td' elements are usually within a 'tr'
                tr_parent = a_tag.find_parent('tr')
                if tr_parent:
                    # Find all 'td' elements within this 'tr' parent
                    td_elements = tr_parent.find_all('td', class_='jsx-3255989801')
                    # Select the second 'td' which is expected to contain the tuition span
                    if len(td_elements) > 1:
                        # The tuition amount is within a span inside the second 'td' element
                        # We use classes to find the correct span
                        tuition_span = td_elements[3].find('span', class_='jsx-3255989801 text-base text-gray')
                        
                        if tuition_span and length_div:
                            tuition = tuition_span.get_text(strip=True)
                            
                            school_info['tuition'] = tuition.replace(';', '')  # Remove semicolons if any
                            
                        else:
                            school_info['tuition'] = 'Not found'
                            

                    length_div = td_elements[3].find('div', class_='jsx-3255989801 mt-2 font-weight-medium text-md text-black text-capitalize')
                    if length_div:
                        length = length_div.get_text(strip=True).split(':')[1] # strip and get the month only
                        school_info['length'] = length.replace(';', '')  #  Remove semicolons if any
                    else:
                        school_info['length'] = 'Not found'

                    td_rank = tr_parent.find('td', class_= 'jsx-3255989801 font-weight-medium text-lg')
                    if td_rank:
                        rank = td_rank.get_text(strip=True)
                        school_info['rank'] = rank.replace('#', '')
                    else:
                        school_info['rank'] = 'Not found'
                all_school_info.append(school_info)
# Print all school information
for info in all_school_info:
    print(info)


Read page_1.html.
Read page_2.html.
Read page_3.html.
Read page_4.html.
{'name': 'Massachusetts Institute of Technology,Cambridge', 'tuition': '$87,600', 'length': '12 months', 'rank': '1'}
{'name': 'University of California,Berkeley', 'tuition': '$27,366', 'length': '1 year', 'rank': '2'}
{'name': 'University of Chicago,Chicago', 'tuition': '$57,105', 'length': '2 years', 'rank': '3'}
{'name': 'University of Pennsylvania,Philadelphia', 'tuition': '$84,874', 'length': '2 years', 'rank': '4'}
{'name': 'Columbia University,New York City', 'tuition': '$33,932', 'length': '1 year', 'rank': '5'}
{'name': 'University of California,Los Angeles', 'tuition': '$33,238', 'length': '15 months', 'rank': '6'}
{'name': 'University of Washington,Seattle', 'tuition': '$32,511', 'length': '1 year', 'rank': '7'}
{'name': 'New York University,New York City', 'tuition': '$54,516', 'length': '1 year', 'rank': '8'}
{'name': 'University of California,San Diego', 'tuition': '$15,102', 'length': '1 year', 'rank

In [70]:
# Specify the CSV file name
csv_file_path = os.path.join(save_directory, 'collegedunia.csv')

# Writing to CSV
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    # Assuming all dictionaries in schools_info have the same structure
    fieldnames = ['name', 'rank', 'length', 'tuition']  # Define the column headers
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()  # Write the column headers

    for school in all_school_info:
        writer.writerow(school)  # Write each school's information as a row

print(f"Data saved to {csv_file_path}.")

Data saved to /Users/xinyutu/Desktop/Homework/DDR/Final Project/collegedunia.csv.
